## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-28-13-04-47 +0000,bbc,US-China talks restart as hopes grow for trade...,https://www.bbc.com/news/articles/cm2197pmlrro
1,2025-07-28-13-01-45 +0000,nyt,Trump Administration Live Updates: President M...,https://www.nytimes.com/live/2025/07/28/us/tru...
2,2025-07-28-13-00-07 +0000,nyt,Citizen Scientists Are Accelerating Ecology Re...,https://www.nytimes.com/2025/07/28/science/cit...
3,2025-07-28-12-52-47 +0000,nyt,"European Carmakers Welcome, and Worry, About U...",https://www.nytimes.com/2025/07/28/us/politics...
4,2025-07-28-12-51-40 +0000,nyt,Is the EU Trade Deal With Trump Good for Europe?,https://www.nytimes.com/2025/07/28/world/europ...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2350/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
9,trump,29
30,deal,17
5,trade,15
16,gaza,12
31,eu,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
15,2025-07-28-12-27-00 +0000,wsj,President Trump said he would shorten the 50-d...,https://www.wsj.com/politics/trump-losing-pati...,105
172,2025-07-27-17-37-25 +0000,nypost,Trump strikes ‘biggest deal ever made’ with EU...,https://nypost.com/2025/07/27/us-news/trump-eu...,81
10,2025-07-28-12-35-00 +0000,wsj,"Stock Market Today: Dollar, Nasdaq Futures Ris...",https://www.wsj.com/livecoverage/stock-market-...,77
4,2025-07-28-12-51-40 +0000,nyt,Is the EU Trade Deal With Trump Good for Europe?,https://www.nytimes.com/2025/07/28/world/europ...,77
145,2025-07-27-20-44-00 +0000,wsj,President Trump said he reached a trade agreem...,https://www.wsj.com/economy/trade/trump-says-h...,74


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
15,105,2025-07-28-12-27-00 +0000,wsj,President Trump said he would shorten the 50-d...,https://www.wsj.com/politics/trump-losing-pati...
40,55,2025-07-28-10-35-00 +0000,wsj,Business leaders on both sides of the Atlantic...,https://www.wsj.com/economy/trade/companies-we...
172,35,2025-07-27-17-37-25 +0000,nypost,Trump strikes ‘biggest deal ever made’ with EU...,https://nypost.com/2025/07/27/us-news/trump-eu...
20,34,2025-07-28-12-03-27 +0000,nyt,Thailand and Cambodia Agree to Halt Fighting T...,https://www.nytimes.com/2025/07/28/world/asia/...
107,33,2025-07-28-00-23-00 +0000,nypost,Thousands of New Yorkers — including ‘Orange i...,https://nypost.com/2025/07/27/us-news/thousand...
192,33,2025-07-27-15-01-21 +0000,bbc,Bowen: Israel's aid measures a gesture to alli...,https://www.bbc.com/news/articles/cz60x5v75p1o
85,24,2025-07-28-04-03-34 +0000,nypost,Cedar Point’s ‘Siren’s Curse’ rollercoaster br...,https://nypost.com/2025/07/28/us-news/cedar-po...
144,24,2025-07-27-20-58-27 +0000,nyt,3 Dead After Small Plane Crashes Off Californi...,https://www.nytimes.com/2025/07/27/us/plane-cr...
55,23,2025-07-28-09-36-19 +0000,nypost,Florida county school board chair deletes comm...,https://nypost.com/2025/07/28/us-news/florida-...
104,22,2025-07-28-01-00-00 +0000,wsj,Individual home buyers are largely locked out ...,https://www.wsj.com/economy/housing/with-indiv...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
